# Permitted Emissions in Minneapolis (MPCA)

This notebook retrieves the Minnesota Pollution Control Agency's annual statewide permitted emissions for the years 2006 - 2021, cleans the entries, clips the dataset to the extent of this project, and saves the results as a csv file.

**Downloaded Data Info:**

CRS: WGS84 - epsg:4326

Size: 38.1mb

**Saved Data Info:**

Saved to a data base in WGS84 (EPSG:4326)

Source: https://www.pca.state.mn.us/air/permitted-facility-air-emissions-data

In [1]:
### Import Libraries

# File manipulation

import os # For working with Operating System
import urllib # For accessing websites
import zipfile # For extracting from Zipfiles
from io import BytesIO # For reading bytes objects

import psycopg2
from psycopg2 import sql, extras

# Analysis

import numpy as np # For working with Arrays
import pandas as pd # Data Manipulation
import arcpy

# Get CWD

cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Create GeoDataBase
# This is the communal GeoDataBase, only run once

if not os.path.exists(os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')): # If it doesn't exist, create it

    arcpy.management.CreateFileGDB(os.path.join(cwd, '..', '..', 'data'), 'QAQC')

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

In [2]:
### Definitions

def extract_zip_from_url(url=None, path=None):
    '''Extract a zipfile from the internet
    then unpack it in to it's own folder 
    within the working directory.
    Takes a single url (string).'''

    if not os.path.exists(path):
        os.mkdir(path)
    # Unload zip into the new folder
    response = urllib.request.urlopen(url) # Get a response
    zip_folder = zipfile.ZipFile(BytesIO(response.read())) # Read Response
    zip_folder.extractall(path=path) # Extract files
    zip_folder.close() # Close zip object

In [3]:
### Download Data
    
## Emissions for all of Minnesota - Downloaded from MPCA (34mb)

url = 'https://files.pca.state.mn.us/pub/file_requests/datasets/Air/PointSourceAirEmissionsInventory.zip'

## Load

# Make folder for files
savepath = os.path.join(cwd, '..','..','data')

extract_zip_from_url(url, savepath)

In [4]:
print('Reading the file. It is in an excel spreadsheet so you may need to install the package: openpyxl\nIt will take a minute or two to read.')

mn_emissions = pd.read_excel(os.path.join(savepath, 'PointSourceAirEmissionsInventory', 'MPCA_PointSourceEmissionInventory.xlsx')) # Load as DataFrame

Reading the file. It is in an excel spreadsheet so you may need to install the package: openpyxl
It will take a minute or two to read.


---

## **Explore the data**

---

In [5]:
# Some observations:
    # This data spans 2006 - 2021
    # There appears to be both HENNEPIN and Hennepin, etc
    # There are also counties called 
        # "Portable Source", "Portable Sources", and "PORTABLE SOURCES" that I'm a little curious about

In [6]:
# Reformat data

focus_columns = ['YEAR', 'FACILITY_ID', 'FACILITY_NAME', 'INDUSTRY_TYPE',
       'NAICS_CODE', 'POLLUTANT', 'LBS_EMITTED', 'COUNTY', 'LATITUDE', 'LONGITUDE']

mn_emissions = mn_emissions.rename(columns={'EMISSIONS (LB)':'LBS_EMITTED'})[focus_columns]

In [8]:
# Keep track of errors
# This is a dictionary

problem_index_collections = {}

In [9]:
# Select for relevant Counties (~ 7 county metro)
# Anoka, Carver, Dakota, Hennepin, Ramsey, Scott, Washington
# The capitalization is weird, but everything's spelt correctly!

mn_emissions['COUNTY'] = mn_emissions['COUNTY'].apply(lambda x : x.upper()) # Get all counties, capitalized

focus_counties = ['ANOKA', 'CARVER', 'DAKOTA', 'HENNEPIN', 'RAMSEY', 'SCOTT', 'WASHINGTON'] # Focus of study

good_index = mn_emissions.COUNTY.isin(focus_counties) # Getting indices of our data

emissions = mn_emissions[good_index] # Subset data

# emissions.shape # 228372 entries

---

## **Group by Facility ID**

For the Facility Table

---

In [52]:
 # Group by all the fields in our Facilities Table
    
    # want only 1 row for each unique facility ID - 926 rows
    
    # Aggregating unique years to see when things change - 2534 rows
    # Also aggregating Industry type because these change within the same year for some reason, too - 2453 rows
    # Name changes happen, going to use the most recent, eventually - 1607 rows
    # NAICS Code really shouldn't change all that much... - 1114 rows
    
    # Which leaves ~200 more entries than we want (differing locational entries)

facilities = emissions.groupby(['FACILITY_ID', 
                                'LATITUDE',
                                'LONGITUDE'], dropna = False).agg({'COUNTY':'unique',
                                                                   'YEAR':'unique', # To be dropped eventually for facility table
                                                                    'INDUSTRY_TYPE':'unique', # Only will use most recent
                                                                    'FACILITY_NAME':'unique', # Only will use most recent
                                                                    'NAICS_CODE':'unique'}) # Only will use most recent

# Ideally after this step all the facilities should have one row, but that isn't the case...

facilities

COUNTY  \
FACILITY_ID LATITUDE  LONGITUDE                  
300003      45.086900 -93.253800       [ANOKA]   
300004      45.078000 -93.275200       [ANOKA]   
300005      45.097788 -93.267733       [ANOKA]   
300010      45.215000 -93.379700       [ANOKA]   
300018      45.048500 -93.270400       [ANOKA]   
...                                        ...   
16300144    44.955900 -92.960300  [WASHINGTON]   
16300147    45.146500 -92.998500  [WASHINGTON]   
            45.146841 -92.998859  [WASHINGTON]   
16300149    45.170886 -92.986449  [WASHINGTON]   
17100103    44.828500 -93.302000    [HENNEPIN]   

                                                                               YEAR  \
FACILITY_ID LATITUDE  LONGITUDE                                                       
300003      45.086900 -93.253800  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...   
300004      45.078000 -93.275200  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...   
300005      45.097788 -93.267733  [2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...   
300010      45.215000 -93.379700  [2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...   
300018      45.048500 -93.270400  [2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...   
...                                                                             ...   
16300144    44.955900 -92.960300  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...   
16300147    45.146500 -92.998500                     [2017, 2018, 2019, 2020, 2021]   
            45.146841 -92.998859                                             [2021]   
16300149    45.170886 -92.986449                                       [2018, 2019]   
17100103    44.828500 -93.302000                     [2017, 2018, 2019, 2020, 2021]   

                                                INDUSTRY_TYPE  \
FACILITY_ID LATITUDE  LONGITUDE                                 
300003      45.086900 -93.253800                      [Other]   
300004      45.078000 -93.275200                      [Other]   
300005      45.097788 -93.267733              [Manufacturing]   
300010      45.215000 -93.379700                      [Other]   
300018      45.048500 -93.270400              [Manufacturing]   
...                                                       ...   
16300144    44.955900 -92.960300              [Manufacturing]   
16300147    45.146500 -92.998500  [Manufacturing, Refineries]   
            45.146841 -92.998859              [Manufacturing]   
16300149    45.170886 -92.986449              [Manufacturing]   
17100103    44.828500 -93.302000              [Manufacturing]   

                                                                      FACILITY_NAME  \
FACILITY_ID LATITUDE  LONGITUDE                                                       
300003      45.086900 -93.253800                 [ISD 14 - Hayes Elementary School]   
300004      45.078000 -93.275200             [ISD 14 - Stevenson Elementary School]   
300005      45.097788 -93.267733                      [Minncast Inc, Minncast Inc.]   
300010      45.215000 -93.379700  [Anoka Metro Regional Treatment Center 4th Ave...   
300018      45.048500 -93.270400  [General Mills Operations Inc - Fridley, Gener...   
...                                                                             ...   
16300144    44.955900 -92.960300        [RR Donnelley Oakdale, RRD - Oakdale Label]   
16300147    45.146500 -92.998500                        [Loadmaster Lubricants LLC]   
            45.146841 -92.998859                        [Loadmaster Lubricants LLC]   
16300149    45.170886 -92.986449                       [Construction Materials Inc]   
17100103    44.828500 -93.302000             [Rainbow Seating LLC, Rainbow Seating]   

                                               NAICS_CODE  
FACILITY_ID LATITUDE  LONGITUDE                            
300003      45.086900 -93.253800          [61111, 611110]  
300004      45.078000 -93.275200          [61111, 611110]  
300005      45.097788 -93.267733       

In [53]:
# Remove multi-index

facilities = facilities.reset_index()

### **Clean Coordinates**

In [54]:
# Are there zeros? 

zero_coords_indices = np.where((facilities.LATITUDE == 0) | 
                      (facilities.LONGITUDE == 0))[0]

problem_index_collections['Zero_coords'] = zero_coords_indices # Store the indices where this occurs

print(len(zero_coords_indices))

# Yes...

facilities.loc[zero_coords_indices].FACILITY_ID.unique()

15


array([ 1900053,  3700267,  3700347,  3700352,  3700353,  5300078,
        5300855,  5301011,  5301034,  5301077, 12300189, 12300760,
       13900041, 13900091, 13900121], dtype=int64)

In [55]:
# Are there Nans?

nan_coords_indices = np.where((np.isnan(facilities.LATITUDE) == True)|
            (np.isnan(facilities.LONGITUDE) == True))[0]

problem_index_collections['Nan_coords'] = nan_coords_indices

# Yes... Oddly enough these are the same facilities as the zeros, just different years

print(len(nan_coords_indices))

facilities.loc[nan_coords_indices].FACILITY_ID.unique()

15


array([ 1900053,  3700267,  3700347,  3700352,  3700353,  5300078,
        5300855,  5301011,  5301034,  5301077, 12300189, 12300760,
       13900041, 13900091, 13900121], dtype=int64)

In [56]:
# Outside of these nans/zeros, which ones are in our study area

nanzero_indices = np.unique(np.concatenate([nan_coords_indices, zero_coords_indices]))

facilities_no_nanzero = facilities.loc[~facilities.index.isin(nanzero_indices)]

## Create bounding box of study area

# Reproject

out_coordinate_system = arcpy.SpatialReference(4326)
mpls_8k_reproject = arcpy.Project_management("mpls_8km", "mpls_8km_wgs.shp", out_coordinate_system)

# Get bounds

mpls_path = os.path.join(savepath, "mpls_8km_wgs.shp")

minx = arcpy.Describe(mpls_path).extent.XMin
maxx = arcpy.Describe(mpls_path).extent.XMax
miny = arcpy.Describe(mpls_path).extent.YMin
maxy = arcpy.Describe(mpls_path).extent.YMax

facilities_no_nanzero_in_study_area = facs = facilities_no_nanzero[(facilities_no_nanzero.LONGITUDE < maxx) &
                                                              (facilities_no_nanzero.LONGITUDE > minx) &
                                                              (facilities_no_nanzero.LATITUDE < maxy)  &
                                                              (facilities_no_nanzero.LATITUDE > miny)]



In [57]:
# Are there discrepancies in their locations?
# facs = facilities without nans/zeros in study area

problem_facility_ids_facs = facs[facs.FACILITY_ID.duplicated(keep='first')].FACILITY_ID

location_problems_facs = facs[facs.FACILITY_ID.duplicated(keep=False)]

# Yes...

location_problems_facs

,FACILITY_ID,LATITUDE,LONGITUDE,COUNTY,YEAR,INDUSTRY_TYPE,FACILITY_NAME,NAICS_CODE
67,300223,45.069700,-93.251400,[ANOKA],"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",[Manufacturing],[Medtronic - Lake Pointe],"[334517, 334510]"
68,300223,45.070591,-93.253136,[ANOKA],"[2012, 2013, 2014, 2015, 2016, 2017, 2018, 201...",[Other],[Medtronic - Lake Pointe],[334510]
82,300245,45.057615,-93.276186,[ANOKA],"[2012, 2013, 2014, 2015, 2016, 2017, 2018, 201...",[Other],"[River Road Industrial Center, BAE Technology ...",[541330]
83,300245,45.058600,-93.276200,[ANOKA],"[2006, 2007, 2009, 2010, 2011, 2012, 2013, 201...",[Utilities],"[BAE Technology Center, River Road Industrial ...","[22133, 541330]"
154,3700037,44.842191,-93.174358,[DAKOTA],"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",[Other],"[Unisys MACS - Eagan, Unisys - Eagan]",[541512]
...,...,...,...,...,...,...,...,...
915,12300680,44.973380,-93.191422,[RAMSEY],"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",[Other],[Brenntag Great Lakes LLC],[424690]
933,12300713,45.046551,-93.151510,[RAMSEY],"[2011, 2012, 2013]",[Other],[Country Insurance & Financial Services],[53112]
934,12300713,45.046969,-93.150933,[RAMSEY],"[2006, 2007, 2008, 2009, 2013]",[Other],[Country Insurance & Financial Services],"[53112, 711310]"
968,12300773,45.065800,-93.220100,[RAMSEY],[2017],[Other],[LaMettry's Collision Inc - New Brighton],[811121]


In [58]:
# How big are the discrepancies in their locations?

distance_discrepancy_indices = []


for fac_id in problem_facility_ids_facs:
    
    fac_data = location_problems_facs[location_problems_facs.FACILITY_ID == fac_id]
    
    done_rows_i = []
    
    for i, row1 in fac_data.iterrows():
        
        pt1 = arcpy.PointGeometry(arcpy.Point(row1.LONGITUDE, row1.LATITUDE),
                                  arcpy.SpatialReference(4626)).projectAs(arcpy.SpatialReference(26915))
        
        done_rows_i += [i]
        
        for j, row2 in fac_data.iterrows():
            
            if j not in done_rows_i:
                
                pt2 = arcpy.PointGeometry(arcpy.Point(row2.LONGITUDE, row2.LATITUDE),
                                          arcpy.SpatialReference(4626)).projectAs(arcpy.SpatialReference(26915))
                
                d = pt1.distanceTo(pt2)
                
                if d > 500: # If the distance difference is greater than 500 meters
                    
                    distance_discrepancy_indices += [i, j]
                
    
facs.loc[distance_discrepancy_indices]

,FACILITY_ID,LATITUDE,LONGITUDE,COUNTY,YEAR,INDUSTRY_TYPE,FACILITY_NAME,NAICS_CODE
339,5300015,45.020100,-93.274400,[HENNEPIN],"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",[Utilities],[Xcel Energy - Riverside Generating Plant],[221112]
340,5300015,45.028404,-93.277496,[HENNEPIN],"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",[Utilities],[Xcel Energy - Riverside Generating Plant],[221112]
524,5300831,44.984315,-93.278424,[HENNEPIN],"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",[Manufacturing],"[Shapco Printing Inc, Shapco Printing]",[323111]
525,5300831,44.990600,-93.356300,[HENNEPIN],"[2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...",[Manufacturing],"[Shapco Printing Inc, Shapco Printing]","[323110, 323111]"


In [59]:
# Well Shapco and Xcel Energy, you've got a discrepency in how you enter the data
# The problem is that these persist throughout the years... Ugh. Add them to the list!

problem_index_collections['Multiple_coords'] = distance_discrepancy_indices

# # We won't Remove them from facs to create clean facilities, but they will be noted as errors

# clean_facilities = facs[~facs.index.isin(distance_discrepancy_indices)]

### Upload to DB

In [61]:
len(facs.FACILITY_ID.unique()) # Number of facilities to upload

446

In [62]:
# Now to get the final dataframe of facilities
# And Upload to Database

# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

# Create Cursor for commands

cur = conn.cursor()

# Insert into table

cols_for_db = ['FACILITY_ID', 'FACILITY_NAME', 'INDUSTRY_TYPE', 'NAICS_CODE', 'COUNTY', 'LAST_REPORT', 'geometry']

cols_for_db = [col.lower() for col in cols_for_db] # Must lowercase our columns...

for fac_id in facs.FACILITY_ID.unique():
    
    current_fac = emissions[emissions.FACILITY_ID == fac_id]
    
    row = [None] * len(cols_for_db)
    
    row[0] = int(fac_id) # Facility ID
    row[5] = int(current_fac.YEAR.max()) # Last report is maximum year
    row[1] = current_fac[current_fac.YEAR == row[5]].FACILITY_NAME.unique()[0] # The name from the last report
    row[2] = str(list(current_fac[current_fac.YEAR == row[5]].INDUSTRY_TYPE.unique())) # industry types from last report
    row[3] = str(list(current_fac[current_fac.YEAR == row[5]].NAICS_CODE.unique())) # NAICS Codes from last report
    row[4] = current_fac[current_fac.YEAR == row[5]].COUNTY.unique()[0] # County
    
    # Get WKT (from most recent year)
    
    lat = current_fac[current_fac.YEAR == row[5]].LATITUDE.unique()[0]
    lon = current_fac[current_fac.YEAR == row[5]].LONGITUDE.unique()[0]
    
    pt = arcpy.PointGeometry(arcpy.Point(lon, lat),
                             arcpy.SpatialReference(4626))
    
    row[6] = pt.WKT
    
        # This is really a great way to insert a lot of data
    
    q = sql.SQL('INSERT INTO MPCA_Facilities ({}) VALUES ({},{});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db)-1)),
     sql.SQL('ST_SetSRID(ST_GeomFromText(%s),4326)::geometry'))
#     print(q.as_string(conn))
#     break

    cur.execute(q.as_string(conn),
        (row)
        )
    # Commit command

    conn.commit()

# Close cursor

cur.close()

# Close connection

conn.close()

## Values

In [91]:
# Get Questionable entries as far as reported emissions

# All emissions of facilities in database
emissions_to_check = emissions[emissions.FACILITY_ID.isin(facs.FACILITY_ID.unique())]

# The combinations of pollutant and facility ID
facility_pollutant_combos = set(emissions_to_check[['FACILITY_ID', 'POLLUTANT']].itertuples(
                                                    index=False, name=None))

# Storage for all questionable entries' index
questionable_zero_indices = np.empty(0, dtype = int)
questionable_change_indices = np.empty(0, dtype = int)

# Error Key Storage
lbs_emitted_errors = {'Questionable Zero':[],
                      'Questionable Change': []}

print(len(facility_pollutant_combos), 'combinations to explore. Takes about 15 minutes')

for i ,facility_pollutant_combo in enumerate(facility_pollutant_combos):
    
    facility_id = facility_pollutant_combo[0]
    pollutant = facility_pollutant_combo[1]
    
    # Do we believe that this combo's values are appropriate?
    
    # Get data sorted by year
    
    facility_pollutant_data = emissions_to_check[(emissions_to_check.FACILITY_ID ==facility_id) &
                                        (emissions_to_check.POLLUTANT ==pollutant)
                                       ].sort_values(by = 'YEAR')

    
    # Important values
    
    years = facility_pollutant_data.YEAR
    lbs_emitted = facility_pollutant_data.LBS_EMITTED
    log_lbs_emitted = np.log10(lbs_emitted) # The magnitude of the emissions
    
    # Year over year change (in magnitude)
    
    delta_mag = np.diff(log_lbs_emitted.values)
    delta_pct = np.diff(lbs_emitted.values)/lbs_emitted[:-1].values * 100 # Percent change from previous year
    
    # Check out the years (after first entry)
    
    # If the magnitude of this pollutant is on average greater than 1 (log10 = 0)
    # Then I think entries should be further vetted
    
    if np.mean(log_lbs_emitted) > 0:
    
        # If the magnitude of this pollutant is on average greater than 1,000 (log10 = 3)
        # Zeros are suspect

        if np.mean(log_lbs_emitted) >= 3:

            zeros = lbs_emitted[1:] == 0 # Zeros (skipping first entry)

            # If the magnitude change was greater than 1 then add it to the list

            big_delta_mag = delta_mag > 1

            zeros_w_big_delta_mag = np.logical_and(zeros, big_delta_mag) 
            
            # Store indices of question

            questionable_years = years[1:][zeros_w_big_delta_mag] # Skipping first entry            
            questionable_zero_indices = np.concatenate((questionable_zero_indices,
                                                        questionable_years.index.values))
            
            # Store Error Key
            if len(questionable_years) > 0:
                lbs_emitted_errors['Questionable Zero'] += [facility_pollutant_combo]
        
        
        # If the change was over 300% from previous year I think that's suspect as well, especially at high magnitudes

        delta300 = abs(delta_pct) > 300 # Pct change greater than 300
        highmags = log_lbs_emitted[1:] > 3 # Magnitude Over 1000
        
        big_change_high_mag = np.logical_and(delta300, highmags) 
        
#         if len(np.unique(big_change_high_mag))>1:
            
#                 print(delta300)
#                 print(years[1:][big_change_high_mag])
#                 print(big_change_high_mag)
#                 print(questionable_years.index.values)
#                 print(facility_pollutant_data)

#                 break

        # Store Indices in question

        questionable_years = years[1:][big_change_high_mag] # Skipping first entry
        questionable_change_indices = np.concatenate((questionable_change_indices,
                                                      questionable_years.index.values))
        
        # Store error key
        if len(questionable_years) > 0:
            lbs_emitted_errors['Questionable Change'] += [facility_pollutant_combo]

21988 combinations to explore. Takes about 15 minutes


C:\Users\HAL\AppData\Local\ESRI\conda\envs\arc_clone\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\HAL\AppData\Local\ESRI\conda\envs\arc_clone\lib\site-packages\numpy\lib\function_base.py:1281: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
<ipython-input-91-f292236ea368>:43: RuntimeWarning: divide by zero encountered in true_divide
  delta_pct = np.diff(lbs_emitted.values)/lbs_emitted[:-1].values * 100 # Percent change from previous year
<ipython-input-91-f292236ea368>:43: RuntimeWarning: invalid value encountered in true_divide
  delta_pct = np.diff(lbs_emitted.values)/lbs_emitted[:-1].values * 100 # Percent change from previous year


In [92]:
len(emissions.loc[questionable_zero_indices]) # No questionable zeros

0

In [93]:
len(emissions.loc[questionable_change_indices]) # A number of questionable changes

613

In [94]:
emissions.loc[questionable_change_indices]

,YEAR,FACILITY_ID,FACILITY_NAME,INDUSTRY_TYPE,NAICS_CODE,POLLUTANT,LBS_EMITTED,COUNTY,LATITUDE,LONGITUDE
470796,2020,5300733,ISD 279 - Osseo Sr High School,Other,61111,Nitrogen Oxides,1.592320e+03,HENNEPIN,45.120162,-93.406306
138633,2011,5300845,Release Coatings of Minneapolis Inc,Manufacturing,332812,Toluene,1.402002e+04,HENNEPIN,44.858400,-93.291400
471798,2020,5300845,Release Coatings of Mpls Inc,Manufacturing,332812,Toluene,9.880023e+03,HENNEPIN,44.858400,-93.291400
107031,2010,5300247,Hard Chrome Inc,Manufacturing,332813,PM Primary,2.703900e+03,HENNEPIN,45.016100,-93.264800
99586,2010,5300293,Fairview Southdale Hospital,Other,62211,Sulfur Dioxide,2.800000e+03,HENNEPIN,44.886100,-93.326800
...,...,...,...,...,...,...,...,...,...,...
205257,2013,300168,Minneapolis Water Works - Fridley,Other,22131,PM Primary,1.058940e+03,ANOKA,45.046376,-93.276679
446224,2020,300168,Minneapolis Water Works - Fridley,Other,22131,PM Primary,1.107260e+03,ANOKA,45.046376,-93.276679
315349,2015,5300084,ADM Milling Co - Nokomis Flour Mill,Manufacturing,311211,CO2-equivalent,1.058000e+06,HENNEPIN,44.939100,-93.231300
331376,2016,5300084,ADM Milling Co - Nokomis Flour Mill,Manufacturing,311211,CO2-equivalent,9.844000e+06,HENNEPIN,44.939100,-93.231300


In [106]:
# Well, I guess we'll just have to trust these companies because this is important data
# Will note in error data, tho

### Upload to DB

In [70]:
# Insert them into database - this takes a while! Like 30 minutes, but I think there's a better way - https://hakibenita.com/fast-load-data-python-postgresql

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

# Create Cursor for commands

cur = conn.cursor()

# Insert into table

cols_for_db = ['FACILITY_ID', 'YEAR', 'POLLUTANT', 'LBS_EMITTED']

cols_for_db_lower = [col.lower() for col in cols_for_db] # Must lowercase our columns...

for i, row in emissions_to_check[cols_for_db].iterrows():
    
        # This is really a great way to insert a lot of data
    
    q = sql.SQL('INSERT INTO MPCA_Permitted_Emissions ({}) VALUES ({});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db_lower)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db))))
#     print(q.as_string(conn))
#     break

    cur.execute(q.as_string(conn),
        (row)
        )
    # Commit command

    conn.commit()

# Close cursor

cur.close()

# Close connection

conn.close()

KeyboardInterrupt: 

### Errors

In [101]:
# Storage for Facilities with errors

error_columns = ['FACILITY_ID', 'Error_codes']

error_datatypes = [int, str]

dtypes = np.dtype(list(zip(error_columns, error_datatypes)))

error_data = pd.DataFrame(np.empty(0, dtype = dtypes))

In [102]:
# Put errors into error dataframe

# Locational issues

for error_code in problem_index_collections:
    
    # Merge bad data with error codes
    
    bad_indices = problem_index_collections[error_code] # Get the bad indices
    bad_data = facilities.loc[bad_indices] # Select the bad data
    bad_data.loc[:,'Error_codes'] = [error_code] * len(bad_data) # Add in an error code column

    error_data = pd.concat([error_data, bad_data[['FACILITY_ID', 'Error_codes']]], ignore_index = True)
    
    
# Value issues

err_fac_ids = []
err_codes = []

for issue in lbs_emitted_errors:
    
    for fac_id, pollutant in lbs_emitted_errors[issue]:
        
        err_fac_ids += [fac_id]
        
        err_codes += [issue + ' - ' + pollutant]
        
# Add these into error_data as well
        
error_data = pd.concat([error_data, 
                       pd.DataFrame(list(zip(err_fac_ids, err_codes)),
                                    columns = ['FACILITY_ID', 'Error_codes'])],
                       ignore_index = True)

In [103]:
len(error_data)

557

In [105]:
# Insert errors into database

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

# Create Cursor for commands

cur = conn.cursor()

# Insert into table

cols_for_db = ['FACILITY_ID', 'Error_codes']

cols_for_db_lower = [col.lower() for col in cols_for_db] # Must lowercase our columns...

for i, row in error_data[cols_for_db].iterrows():
    
        # This is really a great way to insert a lot of data
    
    q = sql.SQL('INSERT INTO MPCA_Facilities_HOLD ({}) VALUES ({});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db_lower)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db))))
#     print(q.as_string(conn))
#     break

    cur.execute(q.as_string(conn),
        (row)
        )
    # Commit command

    conn.commit()

# Close cursor

cur.close()

# Close connection

conn.close()